In [14]:
import tensorflow as tf
import glob

In [19]:
# Get a list of image paths
masks = glob.glob("C:/Users/e_gui/OneDrive/Desktop/data/ISIC2018_Task1_Training_GroundTruth_x2/*.png").sort()
images = glob.glob("C:/Users/e_gui/OneDrive/Desktop/data/ISIC2018_Task1-2_Training_Input_x2/*.png").sort()

In [12]:
def load_image(image):
    # Load the image turn into png
    image = tf.image.decode_png(tf.io.read_file(image_path))
    image_class = tf.image.decode_png(tf.io.read_file(class_path))
    
    # Normalize the images, but not that classes (already between 0-4)
    input_image = tf.cast(image, tf.float32) / 255.0
    
    return image, image_class

None


In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_images, train_classes))